In [2]:
import pandas as pd

In [3]:
atp_matches_2024 = pd.read_csv('atp_matches_2024.csv')
atp_matches_2023 = pd.read_csv('atp_matches_2023.csv')
atp_matches_2022 = pd.read_csv('atp_matches_2022.csv')

atp_matches_qual_chall_2023 = pd.read_csv('atp_matches_qual_chall_2023.csv')
atp_matches_qual_chall_2024 = pd.read_csv('atp_matches_qual_chall_2024.csv')
atp_matches_qual_chall_2022 = pd.read_csv('atp_matches_qual_chall_2022.csv')

all_data = pd.concat([atp_matches_2024,atp_matches_2023, atp_matches_qual_chall_2024, atp_matches_qual_chall_2023, atp_matches_2022, atp_matches_qual_chall_2022], ignore_index=True)

all_data = all_data.dropna(subset=['w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_ace', 'w_ace', 'w_df', 'l_df'])

all_data['tourney_name'] = all_data['tourney_name'].apply(lambda x: 'Davis Cup' if x.startswith('Davis Cup') else x)
all_data = all_data[all_data['tourney_name']!='Davis Cup']
all_data = all_data[all_data['tourney_name']!='United Cup']
all_data = all_data[all_data['surface'].isin(['Hard','Clay','Grass'])]
all_data = all_data[all_data['best_of'] == 3]

In [4]:
num_games = pd.merge(pd.DataFrame(all_data.groupby('winner_name')['tourney_id'].count()),\
                     all_data.groupby('loser_name')['tourney_id'].count(), left_index = True, right_index = True, how ='outer')
num_games = num_games.fillna(0)
num_games['sum'] = num_games['tourney_id_x'] + num_games['tourney_id_y']

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
  print(num_games.sort_values('sum'))

                                      tourney_id_x  tourney_id_y    sum
Justin Lyons                                   0.0           1.0    1.0
Mika Lipp                                      0.0           1.0    1.0
Mikelis Libietis                               0.0           1.0    1.0
Enzo Lima                                      0.0           1.0    1.0
Millen Hurrion                                 0.0           1.0    1.0
Milos Dabic                                    0.0           1.0    1.0
Enmanuel Munoz                                 0.0           1.0    1.0
Min Kyu Song                                   0.0           1.0    1.0
Mischa Zverev                                  0.0           1.0    1.0
Emiliano Aguilera Guerrero                     0.0           1.0    1.0
Mubarak Zaid                                   0.0           1.0    1.0
Elyas Abdulnabi                                0.0           1.0    1.0
Eloi Roux                                      0.0           1.0

In [5]:
#lets investigate data a little bit
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
  print(all_data.groupby('tourney_name')['tourney_id'].count())
#приставка CH вероятно означает challenger

tourney_name
Acapulco                         128
Acapulco CH                       47
Adelaide                          39
Adelaide 1                        82
Adelaide 2                        78
Aguascalientes CH                 46
Aix En Provence CH                75
Aix-En-Provence CH                49
Alicante CH                       96
Ambato CH                         39
Amersfoort CH                     98
Andria CH                         48
Antalya CH                        47
Antofagasta CH                    46
Antwerp                           78
Astana                            82
Astana CH                         48
Asuncion CH                       45
Atlanta                           78
Atp Cup                           54
Auckland                          76
Augsburg CH                       98
Australian Open                  336
Bad Waltersdorf CH                47
Banja Luka                        39
Banja Luka CH                     97
Barcelona                

In [ ]:
def player_statistics(data, name):
  player_stat = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  player_stat[0] = data.loc[data['winner_name'] == name]['w_svpt'].sum() + data.loc[data['loser_name'] == name]['l_svpt'].sum()
  player_stat[1] = data.loc[data['winner_name'] == name]['w_1stIn'].sum() + data.loc[data['loser_name'] == name]['l_1stIn'].sum()
  player_stat[2] = data.loc[data['winner_name'] == name]['w_1stWon'].sum() + data.loc[data['loser_name'] == name]['l_1stWon'].sum()
  player_stat[3] = data.loc[data['winner_name'] == name]['w_2ndWon'].sum() + data.loc[data['loser_name'] == name]['l_2ndWon'].sum()

  player_stat[4] = data.loc[data['winner_name'] == name]['l_svpt'].sum() + data.loc[data['loser_name'] == name]['w_svpt'].sum()
  player_stat[5] = data.loc[data['winner_name'] == name]['l_1stIn'].sum() + data.loc[data['loser_name'] == name]['w_1stIn'].sum()
  player_stat[6] = data.loc[data['winner_name'] == name]['l_1stWon'].sum() + data.loc[data['loser_name'] == name]['w_1stWon'].sum()
  player_stat[7] = data.loc[data['winner_name'] == name]['l_2ndWon'].sum() + data.loc[data['loser_name'] == name]['w_2ndWon'].sum()

  player_stat[8] = data.loc[data['winner_name'] == name]['w_bpFaced'].sum() + data.loc[data['loser_name'] == name]['l_bpFaced'].sum() #сколько было брейкпоинтов в пользу данного игрока
  player_stat[9] = data.loc[data['winner_name'] == name]['w_bpSaved'].sum() + data.loc[data['loser_name'] == name]['l_bpSaved'].sum() #сколько было брейков в пользу данного игрока


  if(player_stat[0] == 0 or player_stat[1] == 0 or player_stat[4] == 0 or player_stat[5] == 0):
    return 'no stat'

  player_stat[10] = player_stat[2]/player_stat[0] # feature 1

  player_stat[11] = player_stat[3]/(player_stat[0] - player_stat[1]) # feature 2

  player_stat[12] = (player_stat[4] - player_stat[6] - player_stat[7])/player_stat[4] #feature 3

  player_stat[13] = player_stat[9]/player_stat[8] #feature 4

  player_stat[14] = len(data.loc[data['winner_name'] == name])/(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #feature 5

  player_stat[15] = (player_stat[2]+player_stat[3]+player_stat[4]-player_stat[6]-player_stat[7])/(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #feature 6


  return player_stat[10:]

In [ ]:
def player_statistics2(data, name):
  player_stat = [0]*19
  player_stat[0] = data.loc[data['winner_name'] == name]['w_svpt'].sum() + data.loc[data['loser_name'] == name]['l_svpt'].sum()
  player_stat[1] = data.loc[data['winner_name'] == name]['w_1stIn'].sum() + data.loc[data['loser_name'] == name]['l_1stIn'].sum()
  player_stat[2] = data.loc[data['winner_name'] == name]['w_1stWon'].sum() + data.loc[data['loser_name'] == name]['l_1stWon'].sum()
  player_stat[3] = data.loc[data['winner_name'] == name]['w_2ndWon'].sum() + data.loc[data['loser_name'] == name]['l_2ndWon'].sum()

  player_stat[4] = data.loc[data['winner_name'] == name]['l_svpt'].sum() + data.loc[data['loser_name'] == name]['w_svpt'].sum()
  player_stat[5] = data.loc[data['winner_name'] == name]['l_1stIn'].sum() + data.loc[data['loser_name'] == name]['w_1stIn'].sum()
  player_stat[6] = data.loc[data['winner_name'] == name]['l_1stWon'].sum() + data.loc[data['loser_name'] == name]['w_1stWon'].sum()
  player_stat[7] = data.loc[data['winner_name'] == name]['l_2ndWon'].sum() + data.loc[data['loser_name'] == name]['w_2ndWon'].sum()

  #performance features

  if(player_stat[0] == 0 or player_stat[1] == 0 or player_stat[4] == 0 or player_stat[5] == 0):
    return 'no stat'

  player_stat[8] = (data.loc[data['winner_name'] == name]['w_ace'].sum() + data.loc[data['loser_name'] == name]['l_ace'].sum()) \
  /(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #число эйсов
  player_stat[9] = (data.loc[data['winner_name'] == name]['w_df'].sum() + data.loc[data['loser_name'] == name]['l_df'].sum()) \
  /(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #число двойных ошибок

  player_stat[10] = (data.loc[data['winner_name'] == name]['w_bpFaced'].sum() + data.loc[data['loser_name'] == name]['l_bpFaced'].sum()) \
  /(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #сколько было брейкпоинтов в пользу данного игрока
  player_stat[11] = (data.loc[data['winner_name'] == name]['w_bpSaved'].sum() + data.loc[data['loser_name'] == name]['l_bpSaved'].sum()) \
  /(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #сколько было брейков в пользу данного игрока

  player_stat[12] = (data.loc[data['winner_name'] == name]['l_bpFaced'].sum() + data.loc[data['loser_name'] == name]['w_bpFaced'].sum()) \
  /(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #сколько было брейкпоинтов в пользу другого игрока
  player_stat[13] = (data.loc[data['winner_name'] == name]['l_bpSaved'].sum() + data.loc[data['loser_name'] == name]['w_bpSaved'].sum()) \
  /(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #сколько было брейков в пользу другого игрока
  #из нейронки кроме брейкпоинта
  player_stat[14] = player_stat[2]/player_stat[0]
  player_stat[15] = player_stat[3]/(player_stat[0] - player_stat[1])
  player_stat[16] = (player_stat[4] - player_stat[6] - player_stat[7])/player_stat[4]
  player_stat[17] = len(data.loc[data['winner_name'] == name])/(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name]))
  player_stat[18] = (player_stat[2]+player_stat[3]+player_stat[4]-player_stat[6]-player_stat[7])/(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name]))

  return player_stat[8:]

In [ ]:
def player_statistics_last(data, name):
  last_data = data.loc[(data['winner_name'] == name) | (data['loser_name'] == name)].sort_values('tourney_date').iloc[-3:] #последние три
  player_stat_last = [0]*19
  player_stat_last[0] = last_data.loc[last_data['winner_name'] == name]['w_svpt'].sum() + last_data.loc[last_data['loser_name'] == name]['l_svpt'].sum()
  player_stat_last[1] = last_data.loc[last_data['winner_name'] == name]['w_1stIn'].sum() + last_data.loc[last_data['loser_name'] == name]['l_1stIn'].sum()
  player_stat_last[2] = last_data.loc[last_data['winner_name'] == name]['w_1stWon'].sum() + last_data.loc[last_data['loser_name'] == name]['l_1stWon'].sum()
  player_stat_last[3] = last_data.loc[last_data['winner_name'] == name]['w_2ndWon'].sum() + last_data.loc[last_data['loser_name'] == name]['l_2ndWon'].sum()
  player_stat_last[4] = last_data.loc[last_data['winner_name'] == name]['l_svpt'].sum() + last_data.loc[last_data['loser_name'] == name]['w_svpt'].sum()
  player_stat_last[5] = last_data.loc[last_data['winner_name'] == name]['l_1stIn'].sum() + last_data.loc[last_data['loser_name'] == name]['w_1stIn'].sum()
  player_stat_last[6] = last_data.loc[last_data['winner_name'] == name]['l_1stWon'].sum() + last_data.loc[last_data['loser_name'] == name]['w_1stWon'].sum()
  player_stat_last[7] = last_data.loc[last_data['winner_name'] == name]['l_2ndWon'].sum() + last_data.loc[last_data['loser_name'] == name]['w_2ndWon'].sum()
  #performance features
  player_stat_last[8] = (last_data.loc[last_data['winner_name'] == name]['w_ace'].sum() + last_data.loc[last_data['loser_name'] == name]['l_ace'].sum()) \
  /(len(data.loc[data['winner_name'] == name])+len(data.loc[data['loser_name'] == name])) #число эйсов
  player_stat_last[9] = (last_data.loc[last_data['winner_name'] == name]['w_df'].sum() + last_data.loc[last_data['loser_name'] == name]['l_df'].sum()) \
  /(len(last_data.loc[last_data['winner_name'] == name])+len(last_data.loc[last_data['loser_name'] == name])) #число двойных ошибок
  if(player_stat_last[0] == 0 or player_stat_last[1] == 0 or player_stat_last[4] == 0 or player_stat_last[5] == 0):
    return 'no stat'

  player_stat_last[10] = (last_data.loc[last_data['winner_name'] == name]['w_bpFaced'].sum() + last_data.loc[last_data['loser_name'] == name]['l_bpFaced'].sum()) \
  /(len(last_data.loc[data['winner_name'] == name])+len(last_data.loc[data['loser_name'] == name])) #сколько было брейкпоинтов в пользу данного игрока
  player_stat_last[11] = (last_data.loc[last_data['winner_name'] == name]['w_bpSaved'].sum() + last_data.loc[last_data['loser_name'] == name]['l_bpSaved'].sum()) \
  /(len(last_data.loc[last_data['winner_name'] == name])+len(last_data.loc[data['loser_name'] == name])) #сколько было брейков в пользу данного игрока

  player_stat_last[12] = (last_data.loc[last_data['winner_name'] == name]['l_bpFaced'].sum() + last_data.loc[last_data['loser_name'] == name]['w_bpFaced'].sum()) \
  /(len(last_data.loc[last_data['winner_name'] == name])+len(last_data.loc[last_data['loser_name'] == name])) #сколько было брейкпоинтов в пользу другого игрока
  player_stat_last[13] = (last_data.loc[last_data['winner_name'] == name]['l_bpSaved'].sum() + last_data.loc[last_data['loser_name'] == name]['w_bpSaved'].sum()) \
  /(len(last_data.loc[last_data['winner_name'] == name])+len(last_data.loc[last_data['loser_name'] == name])) #сколько было брейков в пользу другого игрока
  #из нейронки кроме брейкпоинта
  player_stat_last[14] = player_stat_last[2]/player_stat_last[0]
  player_stat_last[15] = player_stat_last[3]/(player_stat_last[0] - player_stat_last[1])
  player_stat_last[16] = (player_stat_last[4] - player_stat_last[6] - player_stat_last[7])/player_stat_last[4]
  player_stat_last[17] = len(last_data.loc[last_data['winner_name'] == name])/(len(last_data.loc[data['winner_name'] == name])+len(last_data.loc[last_data['loser_name'] == name]))
  player_stat_last[18] = (player_stat_last[2]+player_stat_last[3]+player_stat_last[4]-player_stat_last[6]-player_stat_last[7])/(len(last_data.loc[last_data['winner_name'] == name])+len(last_data.loc[last_data['loser_name'] == name]))
  return player_stat_last[8:]


In [ ]:
#case 1
for ind, row in ml_data.iterrows():
  player1 = player_statistics(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['winner_name'])
  if(player1 == 'no stat'):
    continue
  player2 = player_statistics(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['loser_name'])
  if(player2 == 'no stat'):
    continue
  features = player1 + player2 + [0,0,0]
  if(row['surface'] == 'Hard'):
    features[-3] = 1
  elif(row['surface'] == 'Clay'):
    features[-2] = 1
  else:
    features[-1] = 1

  df_features.append(features)
  y.append(1)
  features = player2 + player1 + [0,0,0]
  if(row['surface'] == 'Hard'):
    features[-3] = 1
  elif(row['surface'] == 'Clay'):
    features[-2] = 1
  else:
    features[-1] = 1
  df_features.append(features)
  y.append(0)


In [ ]:
#case 1 but here i use difference for features of players
for ind, row in ml_data.iterrows():
  player1 = player_statistics(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['winner_name'])
  if(player1 == 'no stat'):
    continue
  player2 = player_statistics(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['loser_name'])
  if(player2 == 'no stat'):
    continue

  current_match = [0,0,0]
  if(row['surface'] == 'Hard'):
    current_match[-3] = 1
  elif(row['surface'] == 'Clay'):
    current_match[-2] = 1
  else:
    current_match[-1] = 1

  df_features.append([a - b for a, b in zip(player1, player2)] + current_match)
  y.append(1)
  df_features.append([a - b for a, b in zip(player2, player1)] + current_match)
  y.append(0)

In [ ]:
#case 2
for ind, row in ml_data.iterrows():
  player1 = player_statistics2(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['winner_name'])
  if(player1 == 'no stat'):
    continue
  player2 = player_statistics2(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['loser_name'])
  if(player2 == 'no stat'):
    continue
  current_match = [0,0,0]
  if(row['surface'] == 'Hard'):
    current_match[-3] = 1
  elif(row['surface'] == 'Clay'):
    current_match[-2] = 1
  else:
    current_match[-1] = 1

  if(row['winner_hand'] == row['loser_hand']):
    current_match +=[1]
  else:
    current_match +=[0]


  df_features.append([a - b for a, b in zip(player1, player2)] + current_match + [row['winner_rank']-row['loser_rank']] + [row['winner_rank_points'] - row['loser_rank_points']])
  y.append(1)
  df_features.append([a - b for a, b in zip(player2, player1)] + current_match + [row['loser_rank']-row['winner_rank']] + [row['loser_rank_points'] - row['winner_rank_points']])
  y.append(0)

In [ ]:
#case 3
#статистика за все время + статистика за последние 3 игры
for ind, row in ml_data.iterrows():
  player1 = player_statistics2(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['winner_name'])
  if(player1 == 'no stat'):
    continue
  player2 = player_statistics2(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['loser_name'])
  if(player2 == 'no stat'):
    continue
  current_match = [0,0,0]
  if(row['surface'] == 'Hard'):
    current_match[-3] = 1
  elif(row['surface'] == 'Clay'):
    current_match[-2] = 1
  else:
    current_match[-1] = 1

  if(row['winner_hand'] == row['loser_hand']):
    current_match +=[1]
  else:
    current_match +=[0]

  player1_last = player_statistics_last(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['winner_name'])
  if(player1 == 'no stat'):
    continue
  player2_last = player_statistics_last(all_data.loc[all_data['tourney_date'] < row['tourney_date']], row['loser_name'])
  if(player2 == 'no stat'):
    continue

  df_features.append([a - b for a, b in zip(player1, player2)] + current_match + [row['winner_rank']-row['loser_rank']] + [row['winner_rank_points'] - row['loser_rank_points']]\
                     + [a - b for a, b in zip(player1_last, player2_last)])
  y.append(1)

  df_features.append([a - b for a, b in zip(player2, player1)] + current_match + [row['loser_rank']-row['winner_rank']] + [row['loser_rank_points'] - row['winner_rank_points']]\
                     + [a - b for a, b in zip(player2_last, player1_last)])
  y.append(0)